# Home assignment 1

You should work on the assignement in groups/teams of 3 participants. 

Upload your solution as a jupyter notebook to moodle by Wednesday, 20th of November 23:59h. (The deadline is strict)
It is sufficient if one student of each team submits the solution.

Do not forget to specify the names of all contributing students in the jupyter notebook.

You should add comments to your code where necessary and print the relevant results. You should also always test your code on self-chosen examples.

# WordNet path similarity

In [4]:
import nltk
from nltk.corpus import wordnet as wn
from collections import deque
import math

def pathSimilarity(word1, word2):
    word_empty = {}
    if word1 in word_empty:
        word1_synsets = word_empty[word1]
    else:
        word1_synsets = wn.synsets(word1)
        word_empty[word1] = word1_synsets
    if word2 in word_empty:
        word2_synsets = word_empty[word2]
    else:
        word2_synsets = wn.synsets(word2)
        word_empty[word2] = word2_synsets
    if not word1_synsets or not word2_synsets:
        return 'enter two words!'
    min_distance = 10000
    for word1_synset in word1_synsets:
        for word2_synset in word2_synsets:
            distance = shortest_distance(word1_synset, word2_synset)
            if distance is not None and distance < min_distance:
                min_distance = distance
    if min_distance == 0:
        return 1.0
    else:
        return (1.0/(min_distance+1))
    
def shortest_distance(word1_synset, word2_synset):   
    if word1_synset == word2_synset:
       return 0
    else:
        distance = float('inf')  #inf stands for no link between two nodes
        dist_dict1, queue1 = {}, deque([(word1_synset, 0)])  #path and to-do list will be searched later
        dist_dict2, queue2 = {}, deque([(word2_synset, 0)])
        while queue1:
            node, depth = queue1.popleft()  #BFS FIFO
            if node in dist_dict1:
                continue
            dist_dict1[node] = depth  #record the searched node and its depth
            for neighbor in node.hypernyms():
                queue1.extend([(neighbor, depth + 1)])  #add neighbor node into to-do list
        while queue2:
            node, depth = queue2.popleft()  #BFS FIFO
            if node in dist_dict2:
                continue
            dist_dict2[node] = depth
            for neighbor in node.hypernyms():
                queue2.extend([(neighbor, depth + 1)])     
        for synset, d1 in dist_dict1.items():
            if synset in dist_dict2:    #no commen synset then infinite path distance
                d2 = dist_dict2[synset]
            else:
                d2 = float('inf')
            distance = min(distance, d1 + d2) #minimal distance between two nodes in two dictionaries from same synset
        if distance == float('inf'):
            return None 
        else:
            return distance

In [5]:
print(pathSimilarity('cat','dog'))
dog = wn.synset('dog.n.01')
cat = wn.synset('cat.n.01')
print(dog.path_similarity(cat))

0.2
0.2


# Markov chain model

In [ ]:
def textNormalization(text):
    pass

In [ ]:
def first_order_markov(training_text, laplace_correction = 0):
    pass

def second_order_markov(training_text, laplace_correction = 0):
    pass

def perplexity (evaluation_text, model):
    pass

# Viterbi Algorithm

In [170]:
import numpy as np
from ast import literal_eval

def Viterbi(State_trans_prob, Word_emission_prob,sentence):
    states = set()
    for x  in list(State_trans_prob.keys()):
        states.add(x[0])
        states.add(x[1])
    states.remove('<s>')
    obs = sentence.split(' ')
    
    # define the problity Matrix of viterbi[N,T],and backpointer
    viterbi = {}
    backpointer = {}
    for state in states:
        for ob in obs:
            viterbi[state,ob] = 0
            backpointer[state,ob] = 0

    # init
    for state in states:
        if (obs[0],state) in Word_emission_prob.keys():
            viterbi[state,obs[0]]=State_trans_prob['<s>',state]*Word_emission_prob[obs[0],state]
        
    # iteration
    for i,ob in enumerate(obs):
        if i > 0:
            for state in states:  
                if(ob,state) in Word_emission_prob.keys():
                    candidate = {}
                    for pre_state in states:
                        if (pre_state,state) in State_trans_prob.keys():
                            candidate[pre_state] = viterbi[pre_state,obs[i-1]] * State_trans_prob[pre_state,state]* Word_emission_prob[ob,state]
                    viterbi[state,ob] = max(candidate.values())
                    
#                     viterbi[state,ob] = max(viterbi[pre_state,obs[i-1]] * State_trans_prob[pre_state,state]* Word_emission_prob[ob,state] \
#                                                 for pre_state in states if (pre_state,state) in State_trans_prob.keys())                
   
                    for key,val in candidate.items():
                        if val == max(candidate.values()):
                            backpointer[state,ob] = key                           
                 
    #terminate
    for key,val in viterbi.items():
        if val == max(viterbi[state,obs[len(obs)-1]]for state in states):
            tag = key[0]
    obs.reverse()
    print(obs)
    
    #backword compute the state for the word in the sentence.
    out={}
    for ob in obs:
        out[ob] = tag
        tag = backpointer[tag,ob]
    final = list(out.values())
    final.reverse()
    
    return final,backpointer,viterbi

# Viterbi(State_trans_prob, Word_emission_prob, Sentence)

In [1]:
State_trans_prob = {('<s>','NNP'):0.2767,('<s>','MD'):0.006,('<s>','VB'):0.0031,('<s>','JJ'):0.0453,('<s>','NN'):0.0449,
                   ('<s>','RB'):0.0510,('<s>','DT'):0.2026,
                   ('NNP','NNP'):0.3777,('NNP','MD'):0.0110,('NNP','VB'):0.0009,('NNP','JJ'):0.0084,('NNP','NN'):0.0584,
                   ('NNP','RB'):0.0090,('NNP','DT'):0.0025,
                   ('MD','NNP'):0.0008,('MD','MD'):0.0002,('MD','VB'):0.7968,('MD','JJ'):0.0005,('MD','NN'):0.0008,
                   ('MD','RB'):0.1698,('MD','DT'):0.0041,
                   ('VB','NNP'):0.0322,('VB','MD'):0.0005,('VB','VB'):0.0050,('VB','JJ'):0.0837,('VB','NN'):0.0615,
                   ('NNP','RB'):0.0514,('NNP','DT'):0.2231,
                   ('JJ','NNP'):0.0366,('JJ','MD'):0.0004,('JJ','VB'):0.0001,('JJ','JJ'):0.0733,('JJ','NN'):0.4509,
                   ('JJ','RB'):0.0036,('JJ','DT'):0.0036,
                   ('NN','NNP'):0.0096,('NN','MD'):0.0176,('NN','VB'):0.0014,('NN','JJ'):0.0086,('NN','NN'):0.1216,
                   ('NN','RB'):0.0177,('NN','DT'):0.0068,
                   ('RB','NNP'):0.0068,('RB','MD'):0.0102,('RB','VB'):0.1011,('RB','JJ'):0.1012,('RB','NN'):0.0120,
                   ('RB','RB'):0.0728,('RB','DT'):0.0479,
                   ('DT','NNP'):0.1147,('DT','MD'):0.0021,('DT','VB'):0.0002,('DT','JJ'):0.2157,('DT','NN'):0.4744,
                   ('DT','RB'):0.0102,('DT','DT'):0.0017}

In [3]:
Word_emission_prob = {('Janet','NNP'):0.000032, ('will','MD'):0.308431,('will','VB'):0.000028,('will','NN'):0.0002,
                     ('back','VB'):0.000672,('back','JJ'):0.00034,('back','NN'):0.000223,('back','RB'):0.010446,
                     ('the','NNP'):0.000048,('the','DT'):0.506099,('bill','VB'):0.000028,('bill','NN'):0.002337}
# assume the missing entries are 0

In [171]:
Sentence = 'Janet will back the bill'
Viterbi(State_trans_prob, Word_emission_prob, Sentence)

['bill', 'the', 'back', 'will', 'Janet']


(['NNP', 'MD', 'RB', 'DT', 'NN'],
 {('NN', 'Janet'): 0,
  ('NN', 'will'): 'NNP',
  ('NN', 'back'): 'MD',
  ('NN', 'the'): 0,
  ('NN', 'bill'): 'DT',
  ('NNP', 'Janet'): 0,
  ('NNP', 'will'): 0,
  ('NNP', 'back'): 0,
  ('NNP', 'the'): 'VB',
  ('NNP', 'bill'): 0,
  ('JJ', 'Janet'): 0,
  ('JJ', 'will'): 0,
  ('JJ', 'back'): 'MD',
  ('JJ', 'the'): 0,
  ('JJ', 'bill'): 0,
  ('DT', 'Janet'): 0,
  ('DT', 'will'): 0,
  ('DT', 'back'): 0,
  ('DT', 'the'): 'RB',
  ('DT', 'bill'): 0,
  ('MD', 'Janet'): 0,
  ('MD', 'will'): 'NNP',
  ('MD', 'back'): 0,
  ('MD', 'the'): 0,
  ('MD', 'bill'): 0,
  ('RB', 'Janet'): 0,
  ('RB', 'will'): 0,
  ('RB', 'back'): 'MD',
  ('RB', 'the'): 0,
  ('RB', 'bill'): 0,
  ('VB', 'Janet'): 0,
  ('VB', 'will'): 'NNP',
  ('VB', 'back'): 'MD',
  ('VB', 'the'): 0,
  ('VB', 'bill'): 'DT'},
 {('NN', 'Janet'): 0,
  ('NN', 'will'): 1.03419392e-10,
  ('NN', 'back'): 5.35925836641536e-15,
  ('NN', 'the'): 0,
  ('NN', 'bill'): 1.4320953590187012e-15,
  ('NNP', 'Janet'): 8.8544e-06,

# POS tagging with nltk

In [168]:
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
def posTag(sentence):
    text = word_tokenize(sentence)
    return nltk.pos_tag(text)


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\unicorn\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


In [169]:
Sentence = 'Janet will back the bill'
posTag(Sentence)

[('Janet', 'NNP'),
 ('will', 'MD'),
 ('back', 'VB'),
 ('the', 'DT'),
 ('bill', 'NN')]